- AWS Documentation: https://docs.aws.amazon.com/bedrock/latest/APIReference/welcome.html
- Bedrock documentation: https://docs.aws.amazon.com/bedrock/latest/userguide/what-is-bedrock.html

AWS Bedrock is a service that allows developers to build and scale generative AI applications by integrating foundational models from various providers. While AWS Bedrock is currently in preview (as of 2025), the steps below outline how to use AWS Bedrock with Python to build AI-powered applications.

---

### **1. Prerequisites**
1. **AWS Account**: Ensure you have access to AWS Bedrock.
2. **Install Boto3**: The Python SDK for AWS.
   ```bash
   pip install boto3
   ```
3. **AWS CLI Configuration**: Ensure you configure AWS CLI with credentials that have permissions to use Bedrock.
   ```bash
   aws configure
   ```

---

### **2. Using AWS Bedrock with Python**
AWS Bedrock enables you to interact with foundation models using the **Boto3** SDK. Below is a step-by-step guide:

---

#### **Step 1: Import Required Libraries**

In [1]:
import boto3
import json
import pandas as pd

---

#### **Step 2: Initialize Bedrock Client**
Create a Bedrock client using **Boto3**:

In [8]:
# Initialize the Bedrock client
bedrock = boto3.client('bedrock', region_name='us-east-1')

---

#### **Step 3: List Available Models**
Check the foundational models (FMs) available through AWS Bedrock:

In [9]:
response = bedrock.list_foundation_models()
models = response.get("modelSummaries", [])

models = pd.DataFrame(models)
models.head()

,modelArn,modelId,modelName,providerName,inputModalities,outputModalities,responseStreamingSupported,customizationsSupported,inferenceTypesSupported,modelLifecycle
0,arn:aws:bedrock:us-east-1::foundation-model/am...,amazon.titan-tg1-large,Titan Text Large,Amazon,[TEXT],[TEXT],True,[],[ON_DEMAND],{'status': 'ACTIVE'}
1,arn:aws:bedrock:us-east-1::foundation-model/am...,amazon.titan-image-generator-v1:0,Titan Image Generator G1,Amazon,"[TEXT, IMAGE]",[IMAGE],NaN,[FINE_TUNING],[PROVISIONED],{'status': 'ACTIVE'}
2,arn:aws:bedrock:us-east-1::foundation-model/am...,amazon.titan-image-generator-v1,Titan Image Generator G1,Amazon,"[TEXT, IMAGE]",[IMAGE],NaN,[],[ON_DEMAND],{'status': 'ACTIVE'}
3,arn:aws:bedrock:us-east-1::foundation-model/am...,amazon.titan-image-generator-v2:0,Titan Image Generator G1 v2,Amazon,"[TEXT, IMAGE]",[IMAGE],NaN,[FINE_TUNING],"[PROVISIONED, ON_DEMAND]",{'status': 'ACTIVE'}
4,arn:aws:bedrock:us-east-1::foundation-model/am...,amazon.titan-text-premier-v1:0,Titan Text G1 - Premier,Amazon,[TEXT],[TEXT],True,[],[ON_DEMAND],{'status': 'ACTIVE'}


#### Filter models

In [49]:
query = lambda name: 'Titan Text Embeddings V2' in name
models[models['modelName'].map(query)]

,modelArn,modelId,modelName,providerName,inputModalities,outputModalities,responseStreamingSupported,customizationsSupported,inferenceTypesSupported,modelLifecycle
20,arn:aws:bedrock:us-east-1::foundation-model/am...,amazon.titan-embed-text-v2:0:8k,Titan Text Embeddings V2,Amazon,[TEXT],[EMBEDDING],False,[],[],{'status': 'ACTIVE'}
21,arn:aws:bedrock:us-east-1::foundation-model/am...,amazon.titan-embed-text-v2:0,Titan Text Embeddings V2,Amazon,[TEXT],[EMBEDDING],False,[],[ON_DEMAND],{'status': 'ACTIVE'}


---

#### **Step 4: Generate Text with a Foundation Model**
Use a foundation model (e.g., text generation, summarization). You need the model ARN or name from the previous step.

In [50]:
import boto3
import json
from botocore.exceptions import ClientError

# Create an Amazon Bedrock Runtime client.
bedrock_runtime = boto3.client(
    'bedrock-runtime',
    region_name='us-east-1',
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY
)

# Set the model ID
model_id = "meta.llama3-70b-instruct-v1:0"  # Example model, replace with a valid one

# Define the prompt for the model.
prompt = "Describe the purpose of a 'hello world' program in one line."

# Format the request payload using the model's native structure.
body = {
    "prompt": prompt,
    "max_gen_len": 30,
    "temperature": 0.5,
    "top_p": 0.9
}

try:
    # Invoke the model with the request.
    response = bedrock_runtime.invoke_model(
        modelId=model_id,
        contentType="application/json",
        accept="application/json",
        body=json.dumps(body),
    )

except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
    exit(1)

# Decode the response body.
model_response = json.loads(response["body"].read())

# Extract and print the response text.
response_text = model_response['generation']
print(response_text)

**
A "hello world" program is a simple program that prints "Hello, World!" to the screen, used to introduce beginners to a new programming


---

#### **Step 5: Use Bedrock for Other Tasks**
AWS Bedrock supports tasks like summarization, classification, and Q&A. Adjust the `body` parameters based on the task. For example, for summarization:

In [ ]:
prompt = "Summarize the following text: 'AWS Bedrock simplifies generative AI workflows ...'"

response = bedrock_runtime.invoke_model(
    modelId=model_id,
    contentType="application/json",
    accept="application/json",
    body=json.dumps({
        "prompt": prompt,
        "task": "summarization"
    })
)

output = json.loads(response['body'])
print("Summary:", output.get("summary", ""))

---

### **3. Permissions Required**
Ensure your IAM user or role has the necessary permissions to access Bedrock:
- `bedrock:InvokeModel`
- `bedrock:ListFoundationModels`
- Any additional permissions based on your task.

For example, an IAM policy might look like this:

In [ ]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "bedrock:ListFoundationModels",
                "bedrock:InvokeModel"
            ],
            "Resource": "*"
        }
    ]
}

---

### **4. Deploying Your Python Application**
You can deploy your Python application using:
1. **AWS Lambda**: For serverless execution.
2. **Amazon ECS/Fargate**: For containerized workloads.
3. **Amazon SageMaker**: If integrating Bedrock into an ML pipeline.

---

### **5. Monitoring and Optimization**
- Use **CloudWatch** to monitor model invocations.
- Optimize model parameters like `max_tokens` and `temperature` for your use case.
- Integrate Bedrock with other AWS services like S3 (data storage) or DynamoDB (for application state management).

---

This approach allows you to leverage AWS Bedrock's foundation models efficiently for various generative AI tasks. Let me know if you'd like more specific examples or guidance!